In [27]:
import re
import json
import pandas as pd

In [28]:
# regex pattern from Bich Ngoc Doan https://github.com/ngoccc/test-analysis/blob/main/explore_misogynistic.ipynb

misogynistic_keywords = [
    # Gendered stereotypes and roles
    r'\b(women|girls)\b.*(drivers?|kitchen|garage|cooking|dishwasher|cleaning|crying|nagging|shopping|emotional|crazy|hysterical|illogical)',
    
    # Comparison-based phrases that demean women
    r'\b(men|man|guys|boys) (better|smarter|stronger|faster|more logical) (than|as) (women|girls)',
    
    # Objectifying language or sexual references
    r'\b(my|your|his|her|our) (girlfriend|wife|woman|daughter)\b.*(body|boobs|butt|legs|vagina|sex|fuck|whore|bitch)',
    
    # "Back to the kitchen" trope
    r'\b(back to the kitchen|stay in the kitchen|only place for women|belong in the kitchen|back in the kitchen)',
    
    # Dismissive phrases about women's logic or abilities
    r'\b(women logic|woman logic|female logic|illogical women|women and logic)',
    
    # Financial dependence or derogatory terms suggesting women are burdens
    r'\b(make you a millionaire|cost me|gold digger|all my money|high maintenance|expensive to keep|drain my wallet)',
    
    # Implicit gendered expectations or objectifying acronyms
    r'\b(W\.I\.F\.E\.)\b|\b(WIFE\b.*(washing|ironing|cleaning|etc))',
    
    # Undermining phrases for women’s achievements or dismissive statements
    r'\b(the reason i preach|feminazi|feminist logic|rape culture)',
    
    # Gendered and dismissive phrases targeting appearance or personality
    r'\b(bitches|hoes|slutty|crazy ex|high maintenance|too emotional|drama queen|gold digger)\b'
]

# Combine all patterns into one regex pattern
misogyny_pattern = re.compile('|'.join(misogynistic_keywords), re.IGNORECASE)

In [29]:
video_meta_data_file = "/Users/renyi/Downloads/dataset/mini-ada-dataset/youniverse-mini/__mini__yt_metadata_en.jsonl.100k"

# read the json line file into a pandas dataframe
df = pd.read_json(video_meta_data_file, lines=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   categories     100000 non-null  object 
 1   channel_id     100000 non-null  object 
 2   crawl_date     100000 non-null  object 
 3   description    100000 non-null  object 
 4   dislike_count  98695 non-null   float64
 5   display_id     100000 non-null  object 
 6   duration       100000 non-null  int64  
 7   like_count     98695 non-null   float64
 8   tags           100000 non-null  object 
 9   title          100000 non-null  object 
 10  upload_date    100000 non-null  object 
 11  view_count     100000 non-null  int64  
dtypes: float64(2), int64(2), object(8)
memory usage: 9.2+ MB


In [30]:
df.head()

,categories,channel_id,crawl_date,description,dislike_count,display_id,duration,like_count,tags,title,upload_date,view_count
0,Film & Animation,UCzWrhkg9eK5I8Bm3HfV-unA,2019-10-31 20:19:26.270363,Lego City Police Lego Firetruck Cartoons about Lego City Movie for kids with a nice long video to keep the kids entertained while learning and having fun with the Lego City characters from Lego City undercover. Thanks for watching!,1.0,SBqSc91Hn9g,1159,8.0,"lego city,lego police,lego city police,lego city episodes,videos de lego city,lego policia,lego bomberos,lego fire truck,lego firetruck,lego police chase,lego robbers,lego cartoons,lego movies,lego videos for kids",Lego City Police Lego Firetruck Cartoons about Lego City Movie for kids Episodes,2016-09-28 00:00:00,1057
1,Film & Animation,UCzWrhkg9eK5I8Bm3HfV-unA,2019-10-31 20:19:26.914516,Lego Marvel SuperHeroes Lego Hulk Smash Iron-Man Lego Spiderman Lego Super Heroes with a nice long almost 1 Hour Movie with the Lego MArvel SuperHeroes characters to keep the little ones entertained while learning and having fun. Thanks for watching!,1.0,UuugEl86ESY,2681,23.0,"Lego superheroes,lego hulk,hulk smash,lego marverl superheroes,lego marvel super heroes movie,lego ironman,lego spiderman,spider-man,lego heroes,lego avengers,lego captain america",Lego Marvel SuperHeroes Lego Hulk Smash Iron-Man Lego Spiderman Lego Super Heroes,2016-09-28 00:00:00,12894
2,Film & Animation,UCzWrhkg9eK5I8Bm3HfV-unA,2019-10-31 20:19:26.531203,Lego City Police Lego Fireman Cartoons about Lego City Movie for kids with another great video about Lego City Police to keep the little ones entertained while learning and having fun. Thanks for watching!,779.0,oB4c-yvnbjs,1394,1607.0,"lego city,lego police,lego city police,lego fire truck,lego cartoons,lego for kids,lego episodes,lego polizi,lego policias,lego police chase,lego policias para niños",Lego City Police Lego Fireman Cartoons about Lego City Movie for kids Episodes,2016-09-28 00:00:00,1800602
3,Film & Animation,UCzWrhkg9eK5I8Bm3HfV-unA,2019-10-31 20:19:28.335329,Lego Harry Potter Complete Lego New Movie for kids Cartoons about Lego with a nice Long80 Minute video to keep the kids and adults entertained enjoying this new movie from Harry Potter and all their adventures. Thanks for watching!,24.0,ZaV-gTCMV8E,5064,227.0,"Lego harry potter,new harry potter,harry potter new movie,movie harry potter,movie harry potter lego,lego harry,lego movies,lego cartoons",Lego Harry Potter Complete Lego New Movie for kids Cartoons about Lego,2016-09-28 00:00:00,57640
4,Film & Animation,UCzWrhkg9eK5I8Bm3HfV-unA,2019-10-31 20:19:30.328487,Lego City Police LONG VIDEO for kids Lego Fire Truck Lego movies with another great video to keep the kids entertained while learning and having fun with the Lego City characters for the little ones to enjoy. Thanks for watching!,13.0,cGvL7AvMfM0,3554,105.0,"lego city,lego police,lego city police,lego fire truck,lego cartoons,cartoons about lego,lego for kids,lego city movie,videos de lego city,lego policias,lego bomberos,lego polizi,polizi lego,lego para niños,lego fireman",Lego City Police 1 HOUR LONG VIDEO for kids Lego Fire Truck Lego movies,2016-09-28 00:00:00,86368


In [31]:
# ref: https://github.com/ngoccc/test-analysis/blob/main/explore_misogynistic.ipynb

# Find misogynistic patterns in title or description
misogynistic_entries = df[df['title'].str.contains(misogyny_pattern) | 
                                 df['description'].str.contains(misogyny_pattern, na=False)]

/var/folders/dx/2ybxmrds0756ksq2t28pxqn00000gn/T/ipykernel_7704/1835870058.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  misogynistic_entries = df[df['title'].str.contains(misogyny_pattern) |
/var/folders/dx/2ybxmrds0756ksq2t28pxqn00000gn/T/ipykernel_7704/1835870058.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['description'].str.contains(misogyny_pattern, na=False)]


In [32]:
misogynistic_entries.info()

<class 'pandas.core.frame.DataFrame'>
Index: 89 entries, 5390 to 99583
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   categories     89 non-null     object 
 1   channel_id     89 non-null     object 
 2   crawl_date     89 non-null     object 
 3   description    89 non-null     object 
 4   dislike_count  88 non-null     float64
 5   display_id     89 non-null     object 
 6   duration       89 non-null     int64  
 7   like_count     88 non-null     float64
 8   tags           89 non-null     object 
 9   title          89 non-null     object 
 10  upload_date    89 non-null     object 
 11  view_count     89 non-null     int64  
dtypes: float64(2), int64(2), object(8)
memory usage: 9.0+ KB


In [33]:
# Set display options for maximum column width
misogynistic_entries[['title', 'description']].sample(frac=1).head()

,title,description
72384,A visit to Elmhurst's Crescent Park neighborhood,"http://gracikmakinneygroup.com/\n\nYesterday I visited two homes in the Crescent Park subdivision in Elmhurst with Tom Makinney and Maria Gracik of Koenig & Strey's Gracik Makinney Group. We were shooting sponsored videos, which will be live at YouTube within a few days.\n\nThe homes, at 350 E Ridge Ave, priced at $439,000, and 511 S Parkview Ave, priced at $1,299,000, illustrate the range of options in Crescent Park. The Ridge home will be open this Sunday, September 18, from 2 to 4 p.m.\n\nThe Crescent Park neighborhood is bounded on the west by Poplar Ave, on the north by St Charles Rd, on the east by I-290 and on the south by the Illinois Prairie Path. A small triangular park, Crescent Park, is at the center of the neighborhood, which has gently curving streets lined with mature trees and a slightly sloping topography. Wild Meadows Park and Washington Park lie along the southern and eastern edges of the area.\n\nAccording to the Elmhurst Historical Museum, Crescent Park was originally a 128-acre farm and was once platted as a cemetery before being sold to a developer who envisioned it as the ""most exclusive residence district"" in Chicago's western suburbs.\n\nToday, says Tom Makinney, whose wife grew up in Crescent Park, it's ""one of the top three subdivisions in Elmhurst."" In the video we stand in Washington Park as Makinney outlines the reasons for his conclusion."
34242,MEGHAN TRAINOR - ALL ABOUT THAT BASS - Easy Piano Tutorial,"Sheet music: https://mnot.es/2yJRMn7\nLearn piano songs quick and easy with this app: https://tinyurl.com/sheetmusic-flowkey\nSUBSCRIBE for a new piano tutorial every day!\n\nLearn to play All About That Bass by Meghan Trainor on piano with this easy piano tutorial! It's simplified so no problem if you're a beginner. Please enjoy.\n\nSheet Music Boss tutorials here: https://www.youtube.com/SheetMusicBoss\n\nFollow Sheet Music Boss!\nhttps://sheetmusicboss.wordpress.com/\nhttps://www.facebook.com/SheetMusicBoss/\nhttps://www.twitter.com/SheetMusicBoss/\nhttps://sheetmusicboss.tumblr.com/\n\nAll About That Bass was the debut single for American singer Meghan Trainor. The song has become a viral success, being spoofed innumerable times and surpassed around 2 billion (2,000,000,000) views on YouTube (counting all uploaded versions of the song). It took Meghan Trainor a lot of work to get the song recorded, approaching various recording labels who were critical of All About That Bass lacking a ""big chorus"" among other things. Regardless of all adversity, Meghan Trainor ultimately released a successful track which has a continuing popularity.\n\nSheet music: https://mnot.es/2yJRMn7\n\nLyrics:\nBecause you know I'm all about that bass,\n'Bout that bass, no treble\nI'm all 'bout that bass, 'bout that bass, no treble\nI'm all 'bout that bass, 'bout that bass, no treble\nI'm all 'bout that bass, 'bout that bass\n\nYeah it's pretty clear, I ain't no size two\nBut I can shake it, shake it like I'm supposed to do\n'Cause I got that boom boom that all the boys chase\nAll the right junk in all the right places\nI see the magazines working that Photoshop\nWe know that shit ain't real\nCome on now, make it stop\nIf you got beauty beauty just raise 'em up\n'Cause every inch of you is perfect\nFrom the bottom to the top\nYeah, my momma she told me don't worry about your size\nShe says, boys they like a little more booty to hold at night\nYou know I won't be no stick-figure, silicone Barbie doll,\nSo, if that's what's you're into\nThen go ahead and move along\n\nBecause you know I'm all about that bass,\n'Bout that bass, no treble\nI'm all 'bout that bass, 'bout that bass, no treble\nI'm all 'bout that bass, 'bout that bass, no treble\nI'm all 'bout that bass, 'bout that bass\n\nI'm bringing booty back\nGo ahead and tell them skinny bitches Hey\nNo, I'm just playing I know you think you're fat,\nBut I'm here to tell you that,\nEvery in

In [43]:
# to jsonlines and id field
misogynistic_entries.reset_index(drop=True, inplace=True)
misogynistic_entries['id'] = misogynistic_entries.index
misogynistic_entries[['id', 'title', 'description']].to_json('regex_misogynistic.json', orient='records', lines=True)


/var/folders/dx/2ybxmrds0756ksq2t28pxqn00000gn/T/ipykernel_7704/3062346689.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  misogynistic_entries['id'] = misogynistic_entries.index
